In [4]:
import cv2
import argparse
import imutils
import time
import scipy.io
# from sklearn.svm import SVC
import skvideo.io
from sklearn.decomposition import PCA
# from sklearn.manifold import Isomap
# from sklearn.manifold import LocallyLinearEmbedding
from scipy.ndimage import morphology as mp
from skimage.feature import canny
from thundersvm import SVC
from random import sample
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction import image
from random import shuffle
import numpy as np
import matplotlib.pyplot as plt
import mahotas
from imutils.video import VideoStream
from imutils.video import FPS
import os
import pickle
import copy

In [2]:
def playVideo(file):
    (major, minor) = cv2.__version__.split(".")[:2]

    OPENCV_OBJECT_TRACKERS = {
        "csrt": cv2.TrackerCSRT_create,
        "kcf": cv2.TrackerKCF_create,
        "boosting": cv2.TrackerBoosting_create,
        "mil": cv2.TrackerMIL_create,
        "tld": cv2.TrackerTLD_create,
        "medianflow": cv2.TrackerMedianFlow_create,
        "mosse": cv2.TrackerMOSSE_create
    }

    # grab the appropriate object tracker using our dictionary of
    # OpenCV object tracker objects
    tracker = OPENCV_OBJECT_TRACKERS["kcf"]()

    vs = cv2.VideoCapture(file)
    initBB = None
    # initialize the FPS throughput estimator
    fps = None
    
    boxes = {}
    
    # loop over frames from the video stream
    fno = 1
    delay=1
    while True:
        # grab the current frame, then handle if we are using a
        # VideoStream or VideoCapture object
        frame1 = vs.read()
        frame = frame1[1] # if args.get("video", False) else frame1
        # frame = fgbg.apply(frame)

        # check to see if we have reached the end of the stream
        if frame is None:
            break

        # resize the frame (so we can process it faster) and grab the
        # frame dimensions
        frame = imutils.resize(frame, width=500)
        (H, W) = frame.shape[:2]

        # check to see if we are currently tracking an object
        if initBB is not None:
            # grab the new bounding box coordinates of the object
            (success, box) = tracker.update(frame)

            # check to see if the tracking was a success
            if success:
                (x, y, w, h) = [int(v) for v in box]
                cv2.rectangle(frame, (x, y), (x + w, y + h),
                    (0, 255, 0), 2)
                boxes[fno]=[x,y,x+w,y+h]

            # update the FPS counter
            if fps is not None:
                fps.update()
                fps.stop()

                # initialize the set of information we'll be displaying on
                # the frame
                info = [
                    ("Tracker", "kcf"),
                    ("Success", "Yes" if success else "No"),
                    ("FPS", "{:.2f}".format(fps.fps())),
                ]
                
                if success:
                    delay=5
                else:
                    delay=10

                # loop over the info tuples and draw them on our frame
                for (i, (k, v)) in enumerate(info):
                    text = "{}: {}".format(k, v)
                    cv2.putText(frame, text, (10, H - ((i * 20) + 20)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        else:
            initBB = cv2.selectROI("Frame", frame, fromCenter=False,
                showCrosshair=True)
            print("Initial BBox: "+str(initBB))

            # start OpenCV object tracker using the supplied bounding box
            # coordinates, then start the FPS throughput estimator as well
            tracker.init(frame, initBB)
            fps = FPS().start()
        # show the output frame
        cv2.imshow("Frame", frame)
        key = cv2.waitKey(10) & 0xFF

        # if the 's' key is selected, we are going to "select" a bounding
        # box to track
        if key == ord("s"):
            # select the bounding box of the object we want to track (make
            # sure you press ENTER or SPACE after selecting the ROI)
            tracker = OPENCV_OBJECT_TRACKERS["kcf"]()
            initBB = None
            # initialize the FPS throughput estimator
            fps = None
            initBB = cv2.selectROI("Frame", frame, fromCenter=False,
                showCrosshair=True)
            print("Frame: "+str(fno)+" Selected BBox: "+str(initBB))

            # start OpenCV object tracker using the supplied bounding box
            # coordinates, then start the FPS throughput estimator as well
            tracker.init(frame, initBB)
            fps = FPS().start()

        # if the `q` key was pressed, break from the loop
        elif key == ord("q"):
            break

        fno=fno+1

    #         # if we are using a webcam, release the pointer
    #         if not args.get("video", False):
    #             vs.stop()

    #         # otherwise, release the file pointer
    #         else:

    vs.release()

    # close all windows
    cv2.destroyAllWindows()
    return boxes

In [2]:
def showBBoxes(file,boxes,label):
    vs = cv2.VideoCapture(file)
    
    # loop over frames from the video stream
    fno = 1
    
    while True:
        # grab the current frame, then handle if we are using a
        # VideoStream or VideoCapture object
        frame1 = vs.read()
        frame = frame1[1] # if args.get("video", False) else frame1
        # frame = fgbg.apply(frame)

        # check to see if we have reached the end of the stream
        if frame is None:
            break

        # resize the frame (so we can process it faster) and grab the
        # frame dimensions
        frame = imutils.resize(frame, width=500)
        (H, W) = frame.shape[:2]

        # check to see if we are currently tracking an object
        if fno in boxes:
            (x1,y1,x2,y2) = boxes[fno]
            cv2.rectangle(frame, (x1, y1), (x2, y2),
                        (0, 255, 0), 2)

        # show the output frame
        cv2.imshow("Class: "+str(label), frame)
        key = cv2.waitKey(20) & 0xFF

#         # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break

        fno=fno+1

    vs.release()

    # close all windows
    cv2.destroyAllWindows()
#     data={'X':X,'Y':Y}
#     return data

In [3]:
########################### Show BBoxes ############################
basePath = "../IndianBirds/"

for i in range(9,10):
    boxes = {}
    fullPath = basePath+str(i)+"/"
    allfiles = os.listdir(fullPath)
    if not allfiles:
        continue
    f = open(fullPath+"bbox.pkl","rb")
#     f = open(fullPath+"bbox.pkl","rb")
    boxes=pickle.load(f)
    f.close()
    j=0;
#     for j in range(2):
#     for j in range(len(allfiles)):
#         file = allfiles[j];
#     box = boxes[allfiles[0]][221]
    for file in allfiles:
        if file.endswith(".mp4"):
            showBBoxes(fullPath+file,boxes[file],i)
            j=j+1
        if (j==2):
            break
#         boxes[file] = b

In [9]:
# svc=SVC()
svc.load_from_file

In [7]:
########################### Track silhouette ###########################
def trackSilhouette(file,boxes,label):
    vs = cv2.VideoCapture(file)
    
    # loop over frames from the video stream
    fno = 1
    x11,x22,y11,y22 = 0,0,0,0
    track = False
        # set up the ROI for tracking
    roi = None
    hsv_roi =  None
    mask = None
    roi_hist = None
    # cv.normalize(roi_hist,roi_hist,0,255,cv.NORM_MINMAX)
    # Setup the termination criteria, either 10 iteration or move by atleast 1 pt
    term_crit = None
    
    while True:
        # grab the current frame, then handle if we are using a
        # VideoStream or VideoCapture object
        frame1 = vs.read()
        frame = frame1[1] # if args.get("video", False) else frame1
        # frame = fgbg.apply(frame)

        # check to see if we have reached the end of the stream
        if frame is None:
            break

        # resize the frame (so we can process it faster) and grab the
        # frame dimensions
        frame = imutils.resize(frame, width=500)
        (H, W) = frame.shape[:2]

        # check to see if we are currently tracking an object
        if fno in boxes:
            (x1,y1,x2,y2) = boxes[fno]
            cropped_im = frame[y1:y2,x1:x2]
            if(track == True):
                hsv = cv2.cvtColor(cropped_im, cv2.COLOR_BGR2HSV)
                dst = cv2.calcBackProject([hsv],[0],roi_hist,[0,180],1)
                # apply meanshift to get the new location
                # print(track_window)
                ret, track_window = cv2.meanShift(dst, track_window, term_crit)
                # Draw it on image
                x11,y11,w,h = track_window
                cv2.rectangle(cropped_im, (x11, y11), (x11+w, y11+h),
                        (0, 255, 0), 2)

            # show the output frame
            cv2.imshow("Class: "+str(label), cropped_im)
            if(track == False):
                next_img = predictSilhouette2(cropped_im)
                sil_gray = cv2.cvtColor(next_img, cv2.COLOR_BGR2GRAY)
                i1, j1 = np.where(sil_gray)
                track = True
                indices = np.meshgrid(np.arange(min(i1), max(i1) + 1),
                                      np.arange(min(j1), max(j1) + 1),
                                      indexing='ij')
                x11,y11 = indices[0][0,0],indices[1][0,0]
                x22,y22 = indices[0][indices[0].shape[0]-1,indices[0].shape[1]-1],indices[1][indices[1].shape[0]-1,indices[1].shape[1]-1]
                track = True
                track_window = (x11,y11,x22,y22)
#                 tracker.init(cropped_im, track_window)
#                 print(initBB)
#                 print('--')
#                 print(track_window)
                roi = next_img
                hsv_roi =  cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
                hsv_im =  cv2.cvtColor(cropped_im, cv2.COLOR_BGR2HSV)
                mask = cv2.inRange(hsv_roi, np.array((0., 0.,0.)), np.array((255.,255.,255.)))
                roi_hist = cv2.calcHist([hsv_im],[0],mask,[180],[0,180])
                cv2.normalize(roi_hist,roi_hist,0,255,cv2.NORM_MINMAX)
                # Setup the termination criteria, either 10 iteration or move by atleast 1 pt
                term_crit = ( cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 100, 1 )
#                     sub_image = cropped_im[indices]
        key = cv2.waitKey(20) & 0xFF

#         # if the `q` key was pressed, break from the loop
        if key == ord("q"):
            break

        fno=fno+1

    vs.release()

    # close all windows
    cv2.destroyAllWindows()
#     data={'X':X,'Y':Y}
#     return data

In [8]:
############################# Method for testing with Strides ###################################
def predictSilhouette2(next_img):
#     x1,y1,x2,y2 = box
    i=0;
    TEST1=None
    psize=8
    svc = SVC()
    svc.load_from_file("training/9/class9_svc.pkl")
    # psize=4
    [m,n,p] = next_img.shape
    while i<(m-psize):
        j=0;
        while j<(n-psize):
            a = copy.copy(next_img[i:i+psize,j:j+psize])
            a = a.flatten()
            if TEST1 is None:
                TEST1 = a
            else:
                TEST1 = np.vstack([TEST1,a])
            j=j+int(psize/2)
        i=i+int(psize/2)

    predictions = svc.predict(TEST1)

    i=0;
    k=0;
    [m,n,p] = next_img.shape
    new_mask = np.zeros([m,n,p])
    count = np.zeros([m,n,p])
    while i<(m-psize):
        j=0;
        while j<(n-psize):
            if k < len(predictions):
                if(predictions[k]==1):
                    new_mask[i:i+psize,j:j+psize] = new_mask[i:i+psize,j:j+psize] + 1
                    count[i:i+psize,j:j+psize] = count[i:i+psize,j:j+psize] + 1
                k = k+1
            j=j+int(psize/2)
        i=i+int(psize/2)

    count [count==0] = 1
    reconstructed1 = []
    reconstructed1 = new_mask/count
    ret,thresh1 = cv2.threshold(reconstructed1,0.7,1,cv2.THRESH_BINARY)
    print("Thresholded Mask:");
#     plt.imshow(thresh1),plt.show();

#     mask1 = reconstructed1.astype(np.bool)
    mask1 = thresh1.astype(np.bool)
    silhouette1 = np.zeros_like(next_img)
    silhouette1[mask1] = next_img[mask1]
    print("Thresholded Silhouette:");
#     plt.imshow(silhouette1),plt.show();

    gray_sil = cv2.cvtColor(silhouette1, cv2.COLOR_BGR2GRAY)
    connected_mask = undesired_objects(gray_sil)
    
    stacked_img = np.stack((connected_mask,)*3, axis=-1)
    
#     plt.imshow(stacked_img),plt.show();
    
    mask = stacked_img.astype(np.bool)
    silhouette = np.zeros_like(next_img)
    silhouette[mask] = next_img[mask]
#     cv2.rectangle(silhouette, (x1, y1), (x2, y2),
#                         (0, 255, 0), 2)
#     cv2.imshow("Frame:",silhouette);
#     plt.imshow(silhouette),plt.show();
    
#     edges = canny(connected_mask)
#     fill_holes = mp.binary_fill_holes(edges)

#     stacked_img = np.stack((fill_holes,)*3, axis=-1)
#     stacked_img = stacked_img.astype(float)
#     # type(stacked_img)
# #     print("Filled Mask:");
# #     plt.imshow(stacked_img),plt.show();

#     mask2 = stacked_img.astype(np.bool)
#     silhouette2 = np.zeros_like(next_img)
#     silhouette2[mask2] = next_img[mask2]
# #     print("Filled Silhouette:");
#     plt.imshow(silhouette2),plt.show();
    
    return silhouette

In [9]:
def undesired_objects (image):
    image = image.astype('uint8')
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(image, connectivity=4)
    sizes = stats[:, -1]

    max_label = 1
    max_size = sizes[1]
    for i in range(2, nb_components):
        if sizes[i] > max_size:
            max_label = i
            max_size = sizes[i]

#     img2 = np.zeros(output.shape)
#     img2[output == max_label] = 255
#     plt.imshow(img2),plt.show();
    img2 = np.zeros(output.shape)
    img2[output == max_label] = 1
    return img2
#     cv2.imshow("Biggest component", img2)
#     cv2.waitKey()

In [10]:
########################### Show BBoxes ############################
basePath = "../IndianBirds/"

for i in range(9,10):
    boxes = {}
    fullPath = basePath+str(i)+"/"
    allfiles = os.listdir(fullPath)
    if not allfiles:
        continue
    f = open(fullPath+"bbox.pkl","rb")
#     f = open(fullPath+"bbox.pkl","rb")
    boxes=pickle.load(f)
    f.close()
    j=0;
#     for j in range(2):
#     for j in range(len(allfiles)):
#         file = allfiles[j];
#     box = boxes[allfiles[0]][221]
    for file in allfiles:
        if file.endswith(".mp4"):
            trackSilhouette(fullPath+file,boxes[file],i)
            j=j+1
        if (j==1):
            break
#         boxes[file] = b

Thresholded Mask:
Thresholded Silhouette:
